In [1]:
import cv2

In [ ]:
pip install dlib

In [2]:
d={'anuhya':0,'madhu':0,'pavitra':0,'prerana':0}


In [3]:
def preprocess_img(img):
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    a,b=gray.shape
    
    new_a=a//2
    new_b=b//2
    
    resized=cv2.resize(gray,(new_b,new_a))
    return resized

In [4]:
def detect_face(img,face_cascade):
    
    faces= face_cascade.detectMultiScale(img,1.1,4)
    if len(faces)==0:
        return None,False
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
    a,b=img.shape
    crop_img = img[y:y+h, x:x+w]
#     cv2.imshow("cropped",crop_img)
    crop_img=cv2.resize(crop_img,(b*2,a*2))
    
    return crop_img,True


In [5]:
face_cascade= cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

In [6]:
# cam=cv2.VideoCapture(0)

# while(True):
#     result,image=cam.read()
#     resized2=preprocess_img(image)
#     detected_img,is_detected = detect_face(resized2,face_cascade)
#     if not is_detected:
#         continue
#     cv2.imshow('frame',detected_img)
#     if cv2.waitKey(1) & 0xFF==ord('q'):
#         cv2.destroyAllWindows()
#         break

# cam.release()
    

In [6]:
source_path='temp'
destination_path = 'anuhya_final'
os.listdir(source_path)

NameError: name 'os' is not defined

In [5]:
import os
import Augmentor

In [6]:
def crop_images(source_folder,destination_folder):
    for i in os.listdir(source_folder): 
        if i not in os.listdir(destination_folder):
            os.mkdir(destination_folder + "/"+i)

        for j in os.listdir(source_folder + "/"+ i):
            img_path = source_folder+'/'+ i+'/'+j

            image=cv2.imread(img_path)
#             cv2.imshow("s",image)
            resized2=preprocess_img(image)

            detected_img,is_detected=detect_face(resized2,face_cascade)
            if is_detected:
                cv2.imwrite(destination_folder+'/'+i+'/'+j, detected_img)



In [10]:
os.listdir(source_folder)

NameError: name 'source_folder' is not defined

In [7]:
# Passing the path of the image directory
def augment(source_folder):
    for i in os.listdir(source_folder):
        if len(os.listdir(source_folder + "/"+i))==0:
            continue 
        p = Augmentor.Pipeline(source_folder + '/'+ i, output_directory=source_folder + "/"+i)

        # Defining augmentation parameters and generating 5 samples
        p.flip_left_right(0.5)
        # p.black_and_white(0.1)
        p.rotate(0.3, 10, 10)
        p.skew(0.4, 0.5)
        p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
        p.sample(100-len(os.listdir(source_folder + "/"+i)))

In [8]:
base_path="C:/facedetect/"
source_folder =base_path+'temp'
destination_folder = base_path+'final'

crop_images(source_folder,destination_folder)


In [9]:
augment(destination_folder)

Processing <PIL.Image.Image image mode=L size=480x848 at 0x25F5BAC7DC0>:   7%|▎   | 2/29 [00:00<00:03,  7.60 Samples/s]

Initialised with 71 image(s) found.
Output directory set to C:/facedetect/final/anuhya.

Processing <PIL.Image.Image image mode=L size=480x848 at 0x25F5AF79D00>: 100%|███| 29/29 [00:00<00:00, 86.35 Samples/s]
Processing <PIL.Image.Image image mode=L size=480x848 at 0x25F5AF55CD0>:  15%|▌   | 6/41 [00:00<00:04,  7.03 Samples/s]

Initialised with 59 image(s) found.
Output directory set to C:/facedetect/final/madhusudhan.

Processing <PIL.Image.Image image mode=L size=480x848 at 0x25F5AECD2E0>: 100%|███| 41/41 [00:00<00:00, 75.16 Samples/s]
Processing <PIL.JpegImagePlugin.JpegImageFile image mode=L size=576x1024 at 0x25F5AF55DF0>: 100%|█| 8/8 [00:00<00:00, 9
Executing Pipeline:   0%|                                                                 | 0/19 [00:00<?, ? Samples/s]

Initialised with 92 image(s) found.
Output directory set to C:/facedetect/final/pavitra.Initialised with 81 image(s) found.
Output directory set to C:/facedetect/final/prerana.

Processing <PIL.Image.Image image mode=L size=480x848 at 0x25F5AE8B7C0>: 100%|███| 19/19 [00:00<00:00, 67.79 Samples/s]


In [10]:
import keras

In [11]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten

In [12]:
classifier = Sequential()

In [13]:
classifier.add(Convolution2D(64,(2,2),input_shape=(64,64,1),activation = 'relu'))
# 64 is the feature detectors, and 2*2 window
# 64,64,1 id the input image size and 1 is gray scale if it was 3 it would be rgb


In [14]:
classifier.add(MaxPooling2D(pool_size=(2,2)))    #size of the sub table ,min is considered = 2*2 

classifier.add(Flatten())

In [15]:
classifier.add(Dense(128,activation = 'relu'))      # first and input layer
classifier.add(Dense(256,activation = 'relu'))      # first and hidden layer
classifier.add(Dense(512,activation = 'relu'))      # second and hidden layer
classifier.add(Dense(4,activation = 'softmax'))     #output=4

In [16]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2, #shearing transformation
        zoom_range=0.1,       # zoom in required
        horizontal_flip=True) # if the images data needs to be horizontally flipped, applicable for real world images

test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_set = train_datagen.flow_from_directory(base_path+"final",
        target_size=(64,64), #size of the image in the model 
        batch_size=8,
        color_mode='grayscale',
        class_mode='categorical')

Found 400 images belonging to 4 classes.


In [19]:
#creates the test set
test_set = test_datagen.flow_from_directory(
        base_path+'test',
        target_size=(64,64),       #size of the image in the model
        batch_size=8,
        color_mode='grayscale',
        class_mode='categorical')

Found 40 images belonging to 4 classes.


In [20]:
classifier.fit_generator(train_set,
                    steps_per_epoch=50,     #number of images
                    epochs=10,
                    validation_data=test_set,
                     validation_steps=5)


<ipython-input-20-cdffd5bf01fc>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(train_set,


Epoch 1/10
50/50 [==============================] - 5s 93ms/step - loss: 1.4352 - accuracy: 0.2925 - val_loss: 1.2853 - val_accuracy: 0.5250
Epoch 2/10
50/50 [==============================] - 4s 84ms/step - loss: 1.1774 - accuracy: 0.4975 - val_loss: 0.8434 - val_accuracy: 0.5000
Epoch 3/10
50/50 [==============================] - 4s 83ms/step - loss: 0.8837 - accuracy: 0.6300 - val_loss: 0.9622 - val_accuracy: 0.5750
Epoch 4/10
50/50 [==============================] - 4s 82ms/step - loss: 0.6307 - accuracy: 0.7600 - val_loss: 0.5296 - val_accuracy: 0.8000
Epoch 5/10
50/50 [==============================] - 4s 83ms/step - loss: 0.4149 - accuracy: 0.8600 - val_loss: 0.6878 - val_accuracy: 0.6750
Epoch 6/10
50/50 [==============================] - 4s 82ms/step - loss: 0.3919 - accuracy: 0.8575 - val_loss: 0.5431 - val_accuracy: 0.7750
Epoch 7/10
50/50 [==============================] - 4s 86ms/step - loss: 0.3723 - accuracy: 0.8525 - val_loss: 0.5733 - val_accuracy: 0.8000
Epoch 8/10
50

In [21]:
train_set.class_indices

{'anuhya': 0, 'madhusudhan': 1, 'pavitra': 2, 'prerana': 3}

In [22]:
classifier.save("model.h5")

In [7]:
import keras
model = keras.models.load_model('model.h5')

In [8]:
import numpy as np

In [9]:
from skimage import transform

In [10]:
def get_image(img):
    resized = preprocess_img(img)
    cropped,is_face=detect_face(resized,face_cascade)
    np_image = np.array(cropped).astype('float32')/255
    np_image = transform.resize(np_image, (64, 64, 1))
    np_image = np.expand_dims(np_image, axis=0) #?
    return np_image

In [12]:
cap = cv2.VideoCapture('VID-20220607-WA0012.mp4') 
count = 0
org = (50, 50)
  
# fontScale
fontScale = 1
   
# Blue color in BGR
color = (255, 0, 0)

font = cv2.FONT_HERSHEY_SIMPLEX
# Line thickness of 2 px
thickness = 2
while True:
    count+=1
    if count%20 !=0:####################################### why
        continue
    # Capture frame-by-frame'
    ret, frame = cap.read()#ret- boolean if any return,cap.read infinite loop

    if not ret:
        break
    np_images=get_image(frame)
    print(list(model.predict(np_images)[0]))
    res = list(model.predict(np_images)[0])
    print(max(res))
    face = res.index(max(res))
    print(face)
    if res[face]>0.9:
        if face ==0:
            frame = cv2.putText(frame, 'Anuhya', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
            d['anuhya']+=1
            if d['anuhya'] >=5:
                send_message('anuhye')
                d['anuhya'] =0
        elif face ==1: 
            frame = cv2.putText(frame, 'Madhu', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
            d['madhu']+=1
            if d['madhu'] >=5:
                send_message('madhu')
                d['madhu'] =0
        elif face ==2:
            frame = cv2.putText(frame, 'Pavitra', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
            d['pavitra']+=1
            if d['pavitra'] >=5:
                send_message('pavitra')
                d['pavitra'] =0
        elif face ==3:
            frame = cv2.putText(frame, 'Prerana', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
            d['prerana']+=1
            if d['prerana'] >=5:
                send_message('prerana')
                d['prerana']=0
#     resized = preprocess_img(frame)
#     cropped,is_face=detect_face(resized,face_cascade)    
#     cropped = get_image(cropped)
#     # Draw a rectangle around the faces
#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
print(d)
cap.release()
cv2.destroyAllWindows()

[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[

nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[0.044491295, 0.81939375, 0.031834822, 0.104280114]
0.81939375
1
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[0.030712498, 0.89063793, 0.017803866, 0.06084568]
0.89063793
1
[0.03616888, 0.8678661, 0.026319012, 0.06964594]
0.8678661
1
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[0.0015372166, 0.0037379095, 0.9867555, 0.007969367]
0.9867555
2
[0.0057085454, 0.0013818443, 0.96136737, 0.031542223]
0.96136737
2
[0.0033675004, 0.0011270

nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[0.67272645, 0.08740888, 0.020385483, 0.21947919]
0.67272645
0
[0.55624, 0.1455899, 0.026805244, 0.2713648]
0.55624
0
[0.13893138, 0.04597636, 0.063357174, 0.7517351]
0.7517351
3
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[nan, nan, nan, nan]
nan
0
[0.5159965, 0.25051156, 0.066539764, 0.16695216]
0.5159965
0
[0.5515287, 0.09167856, 0.037963144, 0.31882954]
0.5515287
0
[0.3819577, 0.02706751, 0.018925304, 0.5720495]
0.5720495
3
[0.27462015, 0.03146968, 0.016840488, 0.6770697]
0.6770697
3
[0.26662627, 0.018359128, 0.013196539, 0.70181805]
0.701818

In [10]:
import cv2

In [6]:
pip install twilio

Note: you may need to restart the kernel to use updated packages.

You should consider upgrading via the 'C:\Users\bbhansali\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
from twilio.rest import Client



In [11]:
def send_message(name):
#     # Your Account SID from twilio.com/console
#     account_sid = "ACc42426d7aded911a0b366b00b6a7b88c"
#     # Your Auth Token from twilio.com/console
#     auth_token  = "e210b29c177792f00f12a9ba44ced8e4"

#     client = Client(account_sid, auth_token)

#     message = client.messages.create(
#         to="+9191482 18777", 
#         from_="+19286159310",
#         body=name + " has entered please take necessary precaution")

#     print(message.sid)
    return